# Exemplo 03: Análise de Enlace
## Identificação de Aeroportos com mais conectividade

Muitos problemas em ciência de dados podem ser modelados com um grafo. Um exemplo é a análise da rede aérea em uma região, onde os vertices são os aeroportos e as areastas são as linhas aéreas. Usando algoritmos de análise de enlace podemos extrair informações como aeroportos mais movimentados e menor caminho entre duas localidades.

Porém essa análise não é tão trivial. A maneira mais simples de determinar o aeroporto mais movimentado é contar o número de voos realizados de e para esta cidade. No entanto, como a maioria das companhias aéreas utiliza um sistema de *hub-and-spoke*, a simples contagem dos voos de entrada e saída não transmite a importância do aeroporto para o tráfego aéreo geral. Isso ocorre porque determinados aeroportos centrais podem ser pontos de passagem para os vôos em outros aeroportos e, como resultado, esses aeroportos centrais podem ser considerados mais importantes, mesmo que tenham contagens total de voos seja igual ou até menores.

O algoritmo Pagerank foi originalmente criado para medir a importância relativa das páginas da web, avaliando os links de ligação da página. Qualquer página da web é considerada mais importante se outras páginas importantes tiverem links para essa página. Podemos aplicar esse mesmo conceito de importância aos aeroportos. Se você substituir “página da web” por “aeroporto” e substituir “link da web” por “voo da companhia aérea”, poderá ver que o PageRank pode ser usado para avaliar a importância de um aeroporto. O PageRank acaba sendo uma boa maneira de medir a importância do aeroporto, dado o uso do molelo *hub-and-spoke* usado pelas companhias hub e spoke das companhias aéreas. Um aeroporto importante acabaria sendo um aeroporto que por si só é um *hub* no qual outros aeroportos possuem muitos vôos ou um “hub de hubs”.

Para este exemplo vamos usar uma base de dados de aeroportos e voos nos Estados Unidos e Canada. Então, para identificar os aeroportos mais importantes da região, considerando como um determinado aeroporto influencia os vôos para outros aeroportos, podemos usar o algoritmo Pagerank. Este exemplo mostra os aeroportos com mais voos e os aeroportos mais conectados calculados pelo Pagerank.

In [1]:
from pyspark.sql import SparkSession

# Import Graphframes lib
from graphframes import GraphFrame

import time
start_time = time.time()

data_path='./data/'

In [2]:
# Cria Spark Session
sc = SparkSession.builder \
    .master("local[*]") \
    .appName("LinkAnalisysAirlines") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.3-spark3.4-s_2.12") \
    .config("spark.jars.repositories", "https://repos.spark-packages.org") \
    .getOrCreate()

https://repos.spark-packages.org added as a remote repository with the name: repo-1
Ivy Default Cache set to: /user/home/marcial/.ivy2/cache
The jars for the packages stored in: /user/home/marcial/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-71a81e71-aea3-49a3-b946-298659b837fb;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found graphframes#graphframes;0.8.3-spark3.4-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 360ms :: artifacts dl 41ms
	:: modules in use:
	graphframes#graphframes;0.8.3-spark3.4-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-71a81e71-aea3-49a3-b946-298659b837fb
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/16ms)
24/04/21 11:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for yo

## Airports

| Field | Description |
| --------- | :-------------: |
| node_id | Unique identifier for the airport. |
| name | Name of airport or city and state.|
| metro_pop | City/region population.|
| latitude | Airport latitude |
| longitude | Airport longitude |


In [3]:
# Le arquivo de dados Airport para Dataframe Spark

airport = sc.read.format("csv").options(sep=',',header='true',inferschema='true').\
         load(data_path+"reachability-meta.csv.gz")

#Exibe campos da tabela e os tipos de dados
#airport.dtypes
print("Airports number: ",airport.count())
airport.show()

Airports number:  456
+-------+--------------------+---------+---------+-----------+
|node_id|                name|metro_pop| latitude|  longitude|
+-------+--------------------+---------+---------+-----------+
|      0|      Abbotsford, BC| 133497.0|49.051575|-122.328849|
|      1|        Aberdeen, SD|  40878.0| 45.45909| -98.487324|
|      2|         Abilene, TX| 166416.0|32.449175| -99.741424|
|      3|    Akron/Canton, OH| 701456.0| 40.79781| -81.371567|
|      4|         Alamosa, CO|   9433.0| 37.46818|-105.873599|
|      5|          Albany, GA| 157688.0| 31.58076| -84.155989|
|      6|          Albany, NY| 871478.0|42.651455| -73.755274|
|      7|     Albuquerque, NM| 898642.0| 35.08418|-106.648639|
|      8|      Alexandria, LA| 154505.0|31.312685| -92.445649|
|      9|Allentown/Bethleh...| 824916.0|40.651428| -75.434219|
|     10|        Alliance, NE|   8499.0| 42.09712|-102.871454|
|     11|          Alpena, MI|  29386.0|45.061565| -83.445154|
|     12|         Altoona, PA| 12

## Flights

| Field | Description |
| --------- | :-------------: |
| FromNodeId | Origin airport (node_id).|
| ToNodeId | Destination airport (node_id).|
| Weight | Distance |


In [4]:
# Le arquivo de dados Linhas Aereas para Dataframe Spark

routes = sc.read.format("csv").options(sep=' ',header='true',inferschema='true').\
          load(data_path+"reachability.txt.gz")

#routes.dtypes
print("Fligths number: ",routes.count())
routes.show()

Fligths number:  71959
+----------+--------+------+
|FromNodeId|ToNodeId|Weight|
+----------+--------+------+
|        27|       0|  -757|
|        57|       0|   -84|
|        70|       0| -1290|
|        74|       0|  -465|
|        86|       0|  -700|
|        94|       0|  -526|
|       100|       0|  -448|
|       113|       0|   -90|
|       138|       0|  -256|
|       154|       0|  -270|
|       166|       0|  -515|
|       178|       0|  -400|
|       230|       0|  -486|
|       235|       0|  -170|
|       242|       0|  -469|
|       246|       0|  -325|
|       262|       0|  -200|
|       269|       0|  -525|
|       275|       0|  -585|
|       280|       0|  -405|
+----------+--------+------+
only showing top 20 rows



## Building Graph

In [5]:
# Extrair campos relevantes para definir os vertices do grafo
vertice = airport.select("node_id","name")

# Graphframe exige que coluna com identificação do vertice possua nome 'id'
# Troca nome "node_id" por "id"
vertice = vertice.withColumnRenamed("node_id", "id").cache()

#caso precise converter algum campo
#vertice = vertice.withColumn("id", vertice["id"].cast("string"))

airport_name = airport.select("node_id", "name").withColumnRenamed("node_id", "id")

#vertice.dtypes
vertice.show(5)

+---+----------------+
| id|            name|
+---+----------------+
|  0|  Abbotsford, BC|
|  1|    Aberdeen, SD|
|  2|     Abilene, TX|
|  3|Akron/Canton, OH|
|  4|     Alamosa, CO|
+---+----------------+
only showing top 5 rows



In [6]:
# Extrair campos relevantes para definir os arestas do grafo
edge = routes.select("FromNodeId","ToNodeId","Weight")

# Graphframe exige que colunas com identificação das arestas possua nome 'src' para origem e 'dst' para destino
# Troca nome "FromNodeId" por "src" e "ToNodeId" por "dst"
edge = edge.withColumnRenamed("FromNodeId", "src") \
           .withColumnRenamed("ToNodeId", "dst").cache()

edge.show(5)

+---+---+------+
|src|dst|Weight|
+---+---+------+
| 27|  0|  -757|
| 57|  0|   -84|
| 70|  0| -1290|
| 74|  0|  -465|
| 86|  0|  -700|
+---+---+------+
only showing top 5 rows



## Create Graph

In [7]:
# Create a GraphFrame graph
g = GraphFrame(vertice, edge)

# Print vertices and edges
g.vertices.show()
g.edges.show()

+---+--------------------+
| id|                name|
+---+--------------------+
|  0|      Abbotsford, BC|
|  1|        Aberdeen, SD|
|  2|         Abilene, TX|
|  3|    Akron/Canton, OH|
|  4|         Alamosa, CO|
|  5|          Albany, GA|
|  6|          Albany, NY|
|  7|     Albuquerque, NM|
|  8|      Alexandria, LA|
|  9|Allentown/Bethleh...|
| 10|        Alliance, NE|
| 11|          Alpena, MI|
| 12|         Altoona, PA|
| 13|        Amarillo, TX|
| 14|       Anchorage, AK|
| 15|        Appleton, WI|
| 16|       Asheville, NC|
| 17|           Aspen, CO|
| 18|          Athens, GA|
| 19|         Atlanta, GA|
+---+--------------------+
only showing top 20 rows



/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+---+---+------+
|src|dst|Weight|
+---+---+------+
| 27|  0|  -757|
| 57|  0|   -84|
| 70|  0| -1290|
| 74|  0|  -465|
| 86|  0|  -700|
| 94|  0|  -526|
|100|  0|  -448|
|113|  0|   -90|
|138|  0|  -256|
|154|  0|  -270|
|166|  0|  -515|
|178|  0|  -400|
|230|  0|  -486|
|235|  0|  -170|
|242|  0|  -469|
|246|  0|  -325|
|262|  0|  -200|
|269|  0|  -525|
|275|  0|  -585|
|280|  0|  -405|
+---+---+------+
only showing top 20 rows



## In-degree of each vertex in the graph

In [8]:
# Aeroportos com mais chegadas

g.inDegrees.join(airport_name, on=['id'], how='inner').select("name", "inDegree").orderBy("inDegree",ascending=False).show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+--------------------+--------+
|                name|inDegree|
+--------------------+--------+
|     Los Angeles, CA|     443|
|   San Francisco, CA|     441|
|Dallas/Fort Worth...|     432|
|       Las Vegas, NV|     429|
|         Chicago, IL|     429|
|        New York, NY|     428|
|          Denver, CO|     427|
|      Washington, DC|     425|
|         Phoenix, AZ|     423|
|  Seattle/Tacoma, WA|     418|
|Minneapolis/St Pa...|     411|
|         Orlando, FL|     407|
|         Toronto, ON|     404|
|    Philadelphia, PA|     403|
|       St. Louis, MO|     401|
|         Houston, TX|     400|
|  Ft. Lauderdale, FL|     400|
|          Boston, MA|     399|
|       San Diego, CA|     397|
|         Detroit, MI|     393|
+--------------------+--------+
only showing top 20 rows



## Triangle Count for each vertice

In [9]:
# Count the number of triagles in each vertice

g.triangleCount().select("name", "count").orderBy("count",ascending=False).show(20)

+--------------------+-----+
|                name|count|
+--------------------+-----+
|     Los Angeles, CA|36684|
|   San Francisco, CA|36483|
|Dallas/Fort Worth...|36283|
|       Las Vegas, NV|36238|
|         Chicago, IL|36078|
|         Phoenix, AZ|35964|
|          Denver, CO|35960|
|  Seattle/Tacoma, WA|35832|
|      Washington, DC|35658|
|Minneapolis/St Pa...|35473|
|    Philadelphia, PA|35274|
|        New York, NY|35188|
|         Atlanta, GA|34886|
|         Detroit, MI|34867|
|         Houston, TX|34740|
|        Portland, OR|34454|
|       Charlotte, NC|34352|
|  Ft. Lauderdale, FL|34268|
|         Memphis, TN|34193|
|          Boston, MA|34072|
+--------------------+-----+
only showing top 20 rows



## Breadth-first search (BFS)

In [10]:
# Run BFS algorithm, and show results. Shortest distance to landmarks 0 and 280

g.shortestPaths(landmarks=["0"]).select("id", "distances").show()

24/04/21 11:14:02 WARN BlockManager: Block rdd_175_0 already exists on this machine; not re-adding it


+---+---------+
| id|distances|
+---+---------+
|315| {0 -> 2}|
|386| {0 -> 2}|
|454| {0 -> 2}|
|365| {0 -> 2}|
|451| {0 -> 1}|
|384| {0 -> 2}|
|324| {0 -> 1}|
|180| {0 -> 2}|
|320| {0 -> 2}|
|373| {0 -> 2}|
|369| {0 -> 2}|
|408| {0 -> 2}|
|307| {0 -> 2}|
|428| {0 -> 2}|
| 11| {0 -> 2}|
| 14| {0 -> 2}|
|346| {0 -> 2}|
| 24| {0 -> 2}|
|302| {0 -> 2}|
|146| {0 -> 2}|
+---+---------+
only showing top 20 rows



## Page Rank

In [11]:
# Run PageRank algorithm, and show results.
pagerank = g.pageRank(resetProbability=0.01, maxIter=20)
#pagerank = g.pageRank(resetProbability=0.01, tol=0.001)

pagerank.vertices.select("name", "pagerank").orderBy("pagerank",ascending=False).show()

+--------------------+------------------+
|                name|          pagerank|
+--------------------+------------------+
|     Los Angeles, CA|2.8123266124650077|
|   San Francisco, CA| 2.800641941117831|
|Dallas/Fort Worth...|2.7460240561317035|
|         Chicago, IL|2.7284968614992158|
|       Las Vegas, NV|2.7185207578102104|
|        New York, NY| 2.717765762525016|
|          Denver, CO| 2.709805369163426|
|      Washington, DC| 2.697910235841779|
|         Phoenix, AZ| 2.685892902162765|
|  Seattle/Tacoma, WA|2.6569722720794258|
|Minneapolis/St Pa...|2.6131209497398773|
|         Orlando, FL|2.5758475670025924|
|         Toronto, ON|2.5538427064279436|
|    Philadelphia, PA| 2.552884494022143|
|         Houston, TX| 2.542357203429309|
|       St. Louis, MO|2.5378217637515204|
|  Ft. Lauderdale, FL| 2.534686007850658|
|          Boston, MA| 2.532035189003591|
|       San Diego, CA| 2.512212403087814|
|         Detroit, MI|2.4958716324639934|
+--------------------+------------

## Stop Spark session

In [12]:
sc.stop()
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

--- Execution time: 19.322293043136597 seconds ---
